In [1]:
from pyspark import SparkContext
sc = SparkContext()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark regression example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [3]:
regressionDataFrame = spark.read.csv('Advertising.csv',header=True, inferSchema = True)

In [4]:
regressionDataFrame = regressionDataFrame.drop('_c0')

In [5]:
regressionDataFrame.take(10)

[Row(TV=230.1, radio=37.8, newspaper=69.2, sales=22.1),
 Row(TV=44.5, radio=39.3, newspaper=45.1, sales=10.4),
 Row(TV=17.2, radio=45.9, newspaper=69.3, sales=9.3),
 Row(TV=151.5, radio=41.3, newspaper=58.5, sales=18.5),
 Row(TV=180.8, radio=10.8, newspaper=58.4, sales=12.9),
 Row(TV=8.7, radio=48.9, newspaper=75.0, sales=7.2),
 Row(TV=57.5, radio=32.8, newspaper=23.5, sales=11.8),
 Row(TV=120.2, radio=19.6, newspaper=11.6, sales=13.2),
 Row(TV=8.6, radio=2.1, newspaper=1.0, sales=4.8),
 Row(TV=199.8, radio=2.6, newspaper=21.2, sales=10.6)]

In [6]:
regressionDataFrame.show(10)

+-----+-----+---------+-----+
|   TV|radio|newspaper|sales|
+-----+-----+---------+-----+
|230.1| 37.8|     69.2| 22.1|
| 44.5| 39.3|     45.1| 10.4|
| 17.2| 45.9|     69.3|  9.3|
|151.5| 41.3|     58.5| 18.5|
|180.8| 10.8|     58.4| 12.9|
|  8.7| 48.9|     75.0|  7.2|
| 57.5| 32.8|     23.5| 11.8|
|120.2| 19.6|     11.6| 13.2|
|  8.6|  2.1|      1.0|  4.8|
|199.8|  2.6|     21.2| 10.6|
+-----+-----+---------+-----+
only showing top 10 rows



In [7]:
type(regressionDataFrame)

pyspark.sql.dataframe.DataFrame

In [8]:
regressionDataRDD = regressionDataFrame.rdd.map(list)

In [9]:
regressionDataRDD.take(5)

[[230.1, 37.8, 69.2, 22.1],
 [44.5, 39.3, 45.1, 10.4],
 [17.2, 45.9, 69.3, 9.3],
 [151.5, 41.3, 58.5, 18.5],
 [180.8, 10.8, 58.4, 12.9]]

In [10]:
from pyspark.mllib.regression import LabeledPoint
regressionDataLabelPoint = regressionDataRDD.map(lambda data : LabeledPoint(data[3], data[0:3]))

In [11]:
regressionDataLabelPoint.take(5)

[LabeledPoint(22.1, [230.1,37.8,69.2]),
 LabeledPoint(10.4, [44.5,39.3,45.1]),
 LabeledPoint(9.3, [17.2,45.9,69.3]),
 LabeledPoint(18.5, [151.5,41.3,58.5]),
 LabeledPoint(12.9, [180.8,10.8,58.4])]

In [12]:
regressionLabelPointSplit = regressionDataLabelPoint.randomSplit([0.7, 0.3])

In [13]:
regressionLabelPointTrainData = regressionLabelPointSplit[0]

In [14]:
regressionLabelPointTrainData.take(5)

[LabeledPoint(22.1, [230.1,37.8,69.2]),
 LabeledPoint(10.4, [44.5,39.3,45.1]),
 LabeledPoint(9.3, [17.2,45.9,69.3]),
 LabeledPoint(7.2, [8.7,48.9,75.0]),
 LabeledPoint(11.8, [57.5,32.8,23.5])]

In [15]:
regressionLabelPointTestData = regressionLabelPointSplit[1]

In [16]:
regressionLabelPointTestData.take(5)

[LabeledPoint(18.5, [151.5,41.3,58.5]),
 LabeledPoint(12.9, [180.8,10.8,58.4]),
 LabeledPoint(10.6, [199.8,2.6,21.2]),
 LabeledPoint(9.2, [23.8,35.1,65.9]),
 LabeledPoint(22.4, [195.4,47.7,52.9])]

In [17]:
from pyspark.mllib.regression import LinearRegressionWithSGD as lrSGD

ourModelWithLinearRegression  = lrSGD.train(data = regressionLabelPointTrainData, 
                                            iterations = 200, step = 0.02, intercept = True)

In [18]:
from pyspark.ml.regression import LinearRegression

# Define LinearRegression algorithm
lr = LinearRegression()

In [19]:
model = lr.fit(regressionLabelPointTrainData)

AttributeError: 'PipelinedRDD' object has no attribute '_jdf'

In [20]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS

model = LogisticRegressionWithLBFGS.train(regressionLabelPointTrainData)

model.weights

Py4JJavaError: An error occurred while calling o171.trainLogisticRegressionModelWithLBFGS.
: org.apache.spark.SparkException: Classification labels should be in [0 to 27]. Found 125 invalid labels.
	at org.apache.spark.ml.classification.LogisticRegression.$anonfun$train$1(LogisticRegression.scala:556)
	at org.apache.spark.ml.util.Instrumentation$.$anonfun$instrumented$1(Instrumentation.scala:191)
	at scala.util.Try$.apply(Try.scala:213)
	at org.apache.spark.ml.util.Instrumentation$.instrumented(Instrumentation.scala:191)
	at org.apache.spark.ml.classification.LogisticRegression.train(LogisticRegression.scala:487)
	at org.apache.spark.mllib.classification.LogisticRegressionWithLBFGS.runWithMlLogisticRegression$1(LogisticRegression.scala:345)
	at org.apache.spark.mllib.classification.LogisticRegressionWithLBFGS.run(LogisticRegression.scala:351)
	at org.apache.spark.mllib.classification.LogisticRegressionWithLBFGS.run(LogisticRegression.scala:317)
	at org.apache.spark.mllib.classification

## Comparisions

In pandas we had `.fit`, and have `.train()` in Pyspark. 

Also, instead of `.predict()` we have `.transform()` in Pyspark

Use [this link](https://spark.apache.org/docs/0.9.0/mllib-guide.html) - we need to pair the actual values with the preds as tuples)

## How to Evaluate the Model (using R^2 or MSE)

- use the `.evaluate() method` to get the metrics

- use the `.summmary()` method on the model object

## How to Use PySpark using LogReg

the summary gives you metrics about the predictions based on the inputs

Steps

1. Get the data
2. scale the data
3. feature engineering 
4. split the data
5. train the model
6. evalulate the model.
7. improve the model
8. try on new data
9. pickle the model to use in production

## Research: Difference between `ml` and `mllib` libraries?

## Great Resource:

https://runawayhorse001.github.io/LearningApacheSpark/pyspark.pdf

## Elephas

This library will let you use PySpark with Keras!

## Alternative to PySparkL H2O

Easier to work with, but more dependencies
Read the docs: http://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/k-means.html